In [20]:
import torch
import pickle
import random
import torchvision
import numpy as np
import datetime as dt
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.training.supervised import GEM, AGEM
from avalanche.logging import InteractiveLogger, TensorboardLogger
from avalanche.benchmarks.datasets import CIFAR100
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceTime, EpochAccuracy

In [21]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
cudnn.enabled = False
cudnn.deterministic = True

In [22]:
train_set = CIFAR100('../Dataset/CIFAR100', train=True, download=True)
test_set = CIFAR100('../Dataset/CIFAR100', train=False, download=True)

transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       )
)

train_set = AvalancheDataset(train_set, transform_groups=transforms_group, initial_transform_group="train")
test_set = AvalancheDataset(test_set, transform_groups=transforms_group, initial_transform_group="eval")

Files already downloaded and verified
Files already downloaded and verified


In [23]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 100
incremental = 20

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=False
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=num_class)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = torch.nn.CrossEntropyLoss()

## GEM

In [24]:
date = dt.datetime.now()
date = date.strftime("%Y_%m_%d_%H_%M_%S")

interactive_logger = InteractiveLogger()
tensor_logger = TensorboardLogger("logs_GEM_cifar100_avalanche_" + date)
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceTime(),
    loggers=[interactive_logger, tensor_logger])

c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\avalanche\training\plugins\evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


In [25]:
train_batch = 10
eval_batch = 10
epoch = 1

strategy = GEM(model, optimizer, criterion, patterns_per_exp=256, memory_strength=0.5, train_epochs=epoch, device=device, train_mb_size=10, evaluator=eval_plugin)

In [26]:
print("Starting experiment...")
results = []

for experience in scenario.train_stream:
    print("Start training on experience ", experience.current_experience)

    strategy.train(experience)
    print("End training on experience ", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(strategy.eval(scenario.test_stream[:]))

# save result
log_path = "./logs.txt"
with open(log_path, 'wb') as lf:
    pickle.dump(results, lf)

Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|██████████| 250/250 [00:05<00:00, 42.06it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2920
-- >> End of training phase << --
End training on experience  0
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 50/50 [00:00<00:00, 127.19it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 1.7904
	Time_Exp/eval_phase/test_stream/Task000/Exp000 = 0.3933
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.4020
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 50/50 [00:00<00:00, 119.38it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 9.7329
	Time_Exp/eval_phase/test_stream/Task000/Exp001 = 0.4188
	Top1_Acc_Exp/eval_phase/test_st